In [1]:
# Full working code to replace LaTeX expressions in the Black-Scholes document

# Import necessary libraries
import re
import os

def convert(input_filename, output_filename):
    # Read the original file
    with open(input_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # First, let's identify all LaTeX commands in the document
    # This will help us understand what we're working with
    latex_commands = re.findall(r'\\[a-zA-Z]+', content)
    unique_commands = sorted(list(set(latex_commands)))

    # Now, let's specifically look at commands starting with d, e, and E
    d_commands = [cmd for cmd in unique_commands if cmd.startswith('\\d')]
    e_commands = [cmd for cmd in unique_commands if cmd.startswith('\\e')]
    E_commands = [cmd for cmd in unique_commands if cmd.startswith('\\E')]


    # Count original occurrences
    original_d_count = content.count('\\d')
    original_e_count = content.count('\\e')
    original_E_count = content.count('\\E')

    print("\
    Original counts:")
    print(f"Occurrences of '\\d': {original_d_count}")
    print(f"Occurrences of '\\e': {original_e_count}")
    print(f"Occurrences of '\\E': {original_E_count}")

    # For each command starting with \d, \e, or \E, count occurrences
    for cmd in d_commands:
        print(f"Occurrences of '{cmd}': {content.count(cmd)}")
    for cmd in e_commands:
        print(f"Occurrences of '{cmd}': {content.count(cmd)}")
    for cmd in E_commands:
        print(f"Occurrences of '{cmd}': {content.count(cmd)}")

    # Now, let's create a new approach using string replacement
    modified_content = content

    # Function to replace only standalone \x (not part of another command)
    def replace_standalone(content, command, replacement):
        # Create a pattern that matches the command when it's not followed by a letter
        # We'll use string operations instead of regex for simplicity
        result = ""
        i = 0
        while i < len(content):
            if content[i:i+len(command)] == command:
                # Check if this is a standalone command (not followed by a letter)
                if i + len(command) >= len(content) or not content[i+len(command)].isalpha():
                    result += replacement
                    i += len(command)
                else:
                    result += command
                    i += len(command)
            else:
                result += content[i]
                i += 1
        return result

    # Replace standalone \d with \mathrm{d}
    modified_content = replace_standalone(modified_content, '\\d', '\\mathrm{d}')

    # Replace standalone \e with \mathrm{e}
    modified_content = replace_standalone(modified_content, '\\e', '\\mathrm{e}')

    # Replace standalone \E with \mathbb{E}
    modified_content = replace_standalone(modified_content, '\\E', '\\mathbb{E}')

    # Count modified occurrences
    mathrm_d_count = modified_content.count('\\mathrm{d}')
    mathrm_e_count = modified_content.count('\\mathrm{e}')
    mathrm_E_count = modified_content.count('\\mathbb{E}')

    # Count remaining occurrences of commands
    modified_d_count = modified_content.count('\\d')
    modified_delta_count = modified_content.count('\\delta')
    modified_e_count = modified_content.count('\\e')
    modified_end_count = modified_content.count('\\end')
    modified_E_count = modified_content.count('\\E')
    modified_EQuotes_count = modified_content.count('\\EQuotes')

    print("\
    After replacements:")
    print(f"Remaining occurrences of '\\d': {modified_d_count}")
    print(f"Occurrences of '\\delta': {modified_delta_count}")
    print(f"Occurrences of '\\mathrm{{d}}': {mathrm_d_count}")
    print(f"Remaining occurrences of '\\e': {modified_e_count}")
    print(f"Occurrences of '\\end': {modified_end_count}")
    print(f"Occurrences of '\\mathrm{{e}}': {mathrm_e_count}")
    print(f"Remaining occurrences of '\\E': {modified_E_count}")
    print(f"Occurrences of '\\EQuotes': {modified_EQuotes_count}")
    print(f"Occurrences of '\\mathrm{{E}}': {mathrm_E_count}")

    # Save the modified content to a new file
    with open(output_filename, 'w', encoding='utf-8') as file:
        file.write(modified_content)

    print(f"\
    Modified content saved to '{output_filename}'")
    print(f"Original file size: {os.path.getsize(input_filename)} bytes")
    print(f"Modified file size: {os.path.getsize(output_filename)} bytes")

    # Show a sample of the replacements
    d_replaced_contexts = []
    for i in range(len(modified_content)):
        if modified_content[i:i+10] == '\\mathrm{d}':
            start = max(0, i-30)
            end = min(len(modified_content), i+40)
            d_replaced_contexts.append(modified_content[start:end])
            if len(d_replaced_contexts) >= 3:
                break

    print("\
    Sample contexts for replaced '\\d' (now '\\mathrm{d}'):")
    for i, context in enumerate(d_replaced_contexts):
        print(f"{i+1}: {context}")

    # Check if we correctly preserved \delta
    delta_contexts = []
    for i in range(len(modified_content)):
        if modified_content[i:i+6] == '\\delta':
            start = max(0, i-30)
            end = min(len(modified_content), i+36)
            delta_contexts.append(modified_content[start:end])
            if len(delta_contexts) >= 3:
                break

    if delta_contexts:
        print("\
    Sample contexts for preserved '\\delta':")
        for i, context in enumerate(delta_contexts):
            print(f"{i+1}: {context}")
    else:
        print("\
    No '\\delta' contexts found in the modified content.")

    # Let's also check for \end
    end_contexts = []
    for i in range(len(modified_content)):
        if modified_content[i:i+4] == '\\end':
            start = max(0, i-30)
            end = min(len(modified_content), i+34)
            end_contexts.append(modified_content[start:end])
            if len(end_contexts) >= 3:
                break

    if end_contexts:
        print("\
    Sample contexts for preserved '\\end':")
        for i, context in enumerate(end_contexts):
            print(f"{i+1}: {context}")
    else:
        print("\
    No '\\end' contexts found in the modified content.")

In [2]:
import glob
import os

# Find all Chapter_*.qmd and Appendix_*.qmd files
chapter_files = glob.glob("Chapter_*.qmd")
appendix_files = glob.glob("Appendix_*.qmd")

all_files = chapter_files + appendix_files

for file in all_files:
    # Create the old filename
    base_name = file.replace('.qmd', '')
    old_filename = base_name + '_old.qmd'
    
    # Rename the original file to _old.qmd
    os.rename(file, old_filename)
    print(f"Renamed {file} to {old_filename}")
    
    # Run convert on the old file to create the new file
    convert(input_filename=old_filename, output_filename=file)
    print(f"Converted {old_filename} to {file}")
    print("---")

Renamed Chapter_Arbitrage.qmd to Chapter_Arbitrage_old.qmd
    Original counts:
Occurrences of '\d': 27
Occurrences of '\e': 3
Occurrences of '\E': 32
Occurrences of '\delta': 26
Occurrences of '\dots': 1
Occurrences of '\e': 3
Occurrences of '\end': 1
Occurrences of '\E': 32
    After replacements:
Remaining occurrences of '\d': 27
Occurrences of '\delta': 26
Occurrences of '\mathrm{d}': 0
Remaining occurrences of '\e': 1
Occurrences of '\end': 1
Occurrences of '\mathrm{e}': 31
Remaining occurrences of '\E': 0
Occurrences of '\EQuotes': 0
Occurrences of '\mathbb{E}': 32
    Modified content saved to 'Chapter_Arbitrage.qmd'
Original file size: 44042 bytes
Modified file size: 43841 bytes
    Sample contexts for replaced '\d' (now '\mathrm{d}'):
    Sample contexts for preserved '\delta':
1:  underlying values; that is, $\delta = (C_u-C_d)/(S_u-S_d)$.  \ind
2: plying by $S_u-S_d$ gives us $\delta(S_u-S_d) = C_u-C_d$ and rearr
3: C_d$ and rearranging yields  $\delta S_u - C_u = \delta S_d